In [79]:
import scipy as sp
import numpy as np
from sympy.abc import x, y
from sympy import Abs
import sympy
from scipy.optimize import minimize

# Метод Ньютона

**Метод Ньютона** – метод поиска минимума функции нескольких переменных. Его особенностью является то, что направление спуска зависит от свойств минимизируемой функции (используются вторые производные).
Используя модели для приближения функции (похожие в окрестности функции), в нашем случае используя ряд Тейлора до второй включительно производной, можно выразить $f_{k + 1}$ через $f_k$ и при условии минимума получить формулу для шага:
\begin{align}
p_{(k)} = hessian(f_{(k)})^{-1} \cdot grad(f_{(k)})
\end{align}

Для получения градиента использовалась функция *sp.optimize.approx_fprime*.

Для получения гессиана использовалась функция *sympy.hessian*.

## Используемые методы одномерной оптимизации:
1. **Метод золотого сечения** (см. metopt1)
2. **Шаг, длина которого зависит от свойств минимизируемой функции**
Можно не менять направление до тех пор, пока функция $f(x)$ в направлении $p_{(k)}$ убывает. Тогда будем двигаться по направлению убывания до его минимума – точки x_{(k + 1)}. В ней же определим новое направление движения, ортогональное текущему.
С помощью моделей для приближения функции получаем следующую формулу для длины шага:
\begin{align}
\alpha = - \frac {grad(f_{(k)})^{Τ} \cdot p_{(k)}} { p_{(k)}^{T} \cdot hessian(f_{(k)}) \cdot p_{(k)}}
\end{align}

3. **Правило Вольфе**   
Возьмем касательную в нашей точке и приподнимем ее на коэффициент $c_1$, значение которого около единицы, но меньше ее. Полученная прямая предположительно пересекает нашу функцию в точке, в которой значение функции уже возрастает. Следовательно, искомая следующая точка лежит в этом промежутке.  
Возьмем теперь касательную с коэффициентом $c_2$, большим $0$, но около него.Заданная точка предположительно должна находится на промежутке убывания функции, однако уже ближе к минимуму, и, следовательно, с более ‘пологой’ касательной. Итого, наклон касательной в текущей точке должен быть резче чем полученный наклон.
Эти два рассуждения задают два условия.  
1) Условие Армихо
\begin{align}
f(x_{(k)} + α \cdot p_{(k)})  - f(x_{(k)}) \le c1 \cdot \alpha \cdot \langle grad( f(x_{(k)}) ),  p_{(k)}\rangle
\end{align}
2) Условие кривизны
\begin{align}
\langle grad( f(x_{(k)} + α \cdot p_{(k)})),  p_{(k)}\rangle \ge c_2 \cdot \langle grad( f(x_{(k)}) ),  p_{(k)}\rangle
\end{align}




In [80]:
def constant_step(step_size, x_last, p, f_runnable, eps, gradient, hessian):
    '''The function returns a constant specified step'''
    return np.array([step_size, 0, 0]);

def golden_step(step_size, x_last, p, f_runnable, eps, gradient, hessian):
    '''The function returns the step calculated using the golden ratio method'''
    phi = (1 + 5 ** 0.5) / 2
    cnt_func = 0
    a = 0
    b = 1
    ml = b - (b - a) / phi
    mr = a + (b - a) / phi
    xl = [x_i + ml * p_i[0] for x_i, p_i in zip(x_last, p)]
    xr = [x_i + mr * p_i[0] for x_i, p_i in zip(x_last, p)]
    lvalue = f_runnable(xl)
    rvalue = f_runnable(xr)
    cnt_func += 2

    while (b - a > eps):
      if lvalue < rvalue:
        b = mr
        mr = ml
        ml = b - (b - a) / phi
        rvalue = lvalue
        xr = xl
        xl = [x_i + ml * p_i[0] for x_i, p_i in zip(x_last, p)]
        lvalue = f_runnable(xl)
        cnt_func += 1
      else:
        a = ml
        ml = mr
        mr = a + (b - a) / phi
        lvalue = rvalue
        xl = xr
        xr = [x_i + mr * p_i[0] for x_i, p_i in zip(x_last, p)]
        rvalue = f_runnable(xr)
        cnt_func += 1
    return np.array([a, cnt_func, 0]);

def best_step(step_size, x_last, p, f_runnable, eps, gradient, hessian):
  '''The function returns a step whose value depends on the properties of the minimized function'''
  numerator = np.dot(gradient.transpose(), p)
  denominator = np.dot(np.dot(p.transpose(), hessian), p)
  res = -1 * numerator[0][0] / denominator[0][0]
  return np.array([res, 0, 0]);

def step_Wolfe_rule(step_size, x_last, p, f_runnable, eps, gradient, hessian):
  '''The function returns the step calculated using Wolfe's rule'''
  e1 = 0.8
  e2 = 0.2
  l = 0.8
  a = 1
  cnt_func = 1
  cnt_grad = 0
  f_last = f_runnable(x_last)

  x_new = [x + a * p_i[0] for x, p_i in zip(x_last, p)]
  diff1 = f_runnable(x_new) - f_last
  cnt_func += 1

  gradient_x_new = sp.optimize.approx_fprime(x_new, f_runnable, 1e-6).reshape((2, 1))
  diff2 = np.dot(np.squeeze(np.asarray(gradient_x_new)), np.squeeze(np.asarray(p)))
  cnt_grad += 1

  tangent = np.dot(np.squeeze(np.asarray(gradient)), np.squeeze(np.asarray(p)))
  l1 = e1 * a * tangent
  l2 = e2 * tangent

  while ((diff1 - l1 > eps) and (diff2 - l2 >= eps)):
    a = l * a

    x_new = [x + a * p_i[0] for x, p_i in zip(x_last, p)]
    diff1 = f_runnable(x_new) - f_last
    cnt_func += 1

    gradient_x_new = sp.optimize.approx_fprime(x_new, f_runnable, 1e-6).reshape((2, 1))
    diff2 = np.dot(np.squeeze(np.asarray(gradient_x_new)), np.squeeze(np.asarray(p)))
    cnt_grad += 1

    tangent = np.dot(np.squeeze(np.asarray(gradient)), np.squeeze(np.asarray(p)))
    l1 = e1 * a * tangent
    l2 = e2 * tangent

  return np.array([a, cnt_func, cnt_grad]);

In [81]:
def gradient_descent_golden_ratio(f, x0, eps):
  x_last = x0
  x_res = x0
  phi = (1 + 5 ** 0.5) / 2
  cnt_grad = 0
  cnt_func = 0
  cnt_iter = 0

  while True:
    cnt_iter += 1

    cnt_grad += 1
    grad = sp.optimize.approx_fprime(x_last, f, 1e-6)

    a = 0
    b = 1
    ml = b - (b - a) / phi
    mr = a + (b - a) / phi
    xl = [x - ml * grad_i for x, grad_i in zip(x_last, grad)]
    xr = [x - mr * grad_i for x, grad_i in zip(x_last, grad)]
    lvalue = f(xl)
    rvalue = f(xr)
    cnt_func += 2

    while (b - a > eps):
      if lvalue < rvalue:
        b = mr
        mr = ml
        ml = b - (b - a) / phi
        rvalue = lvalue
        xr = xl
        xl = [x - ml * grad_i for x, grad_i in zip(x_last, grad)]
        lvalue = f(xl)
        cnt_func += 1
      else:
        a = ml
        ml = mr
        mr = a + (b - a) / phi
        lvalue = rvalue
        xl = xr
        xr = [x - mr * grad_i for x, grad_i in zip(x_last, grad)]
        rvalue = f(xr)
        cnt_func += 1

    x_new = [x - a * grad_i for x, grad_i in zip(x_last, grad)]

    cnt_func += 2
    if abs(f(x_new) - f(x_last)) < eps:
      break;

    x_last = x_res
    x_res = x_new

  res = np.array([x_res[0], x_res[1], f(x_res), cnt_iter, cnt_grad, cnt_func])
  return res;

In [82]:
import numpy as np
from numpy.linalg import LinAlgError

def Newton_method(f_runnable, f_sympy, f_step, x0, eps, step_size):
    """Newton method.

    Returns a numpy array containing:
    the x coordinates (at which the minimum is reached),
    the minimum value,
    the number of iterations,
    the number of gradient and hessian calculations,
    the number of function calculations.
    """
    x_res = x0
    x_last = x0
    cnt_grad_and_hess = 0
    cnt_func = 0
    cnt_iter = 0
    hess = sympy.hessian(f_sympy, (x, y))

    while True:
      cnt_iter += 1

      cnt_grad_and_hess += 1
      gradient = sp.optimize.approx_fprime(x_last, f_runnable, 1e-6).reshape((len(x0), 1))
      hessian = hess.subs(x, x_last[0]).subs(y, x_last[1])

      if hessian.det() ==  0.0:
        print("Erorr: Matrix det == 0. Current function: ",  f_sympy, ", current arguments: ", x_last[0], x_last[1])
        print("Calculations continue using gradient descent.")
        print()
        return gradient_descent_golden_ratio(f_runnable, x_last, eps);
      else:
        p = np.asarray(-((hessian ** - 1) * gradient))

      eval = f_step(step_size, x_last, p, f_runnable, eps, gradient, hessian)
      step = eval[0]
      cnt_func += eval[1]
      cnt_grad_and_hess += eval[2]
      x_new = [x_i + step * p_i[0] for x_i, p_i in zip(x_last, p)]

      cnt_func += 2
      if abs(f_runnable(x_new) - f_runnable(x_last)) < eps:
        break;

      x_last = x_res
      x_res = x_new

    res = np.array([*x_res, f_runnable(x_res), cnt_iter, cnt_grad_and_hess, cnt_func])
    return res;

In [83]:
import autograd.numpy as anp
from autograd import grad, jacobian

# Nelder-Mead
def Nelder_Mead_from_scipy_optimize(f_runnable, x0, eps):
   result = minimize(f_runnable, x0, method='Nelder-Mead')
   return np.array([*result.x, result.fun, result.nit, 0, result.nfev])

# Метод Ньютона с постоянным шагом
def Newton_constant_step_method(f_runnable, f_sympy, x0, step_size, eps):
    return Newton_method(f_runnable, f_sympy, constant_step, x0, eps, step_size)

# Метод Ньютона с одномерным поиском (метод золотого сечения)
def Newton_golden_ratio_method(f_runnable, f_sympy, x0, eps):
    return Newton_method(f_runnable, f_sympy, golden_step, x0, eps, 0)

# Метод Ньютона с шагом, длина которого зависит от свойств минимизируемой функции
def Newton_best_step_method(f_runnable, f_sympy, x0, eps):
    return Newton_method(f_runnable, f_sympy, best_step, x0, eps, 0)

# Метод Ньютона реализованный в библиотеке scipy.optimize
def Newton__method_from_scipy_optimize(f_runnable, x0, eps):
   jacobian_f = jacobian(f_runnable)
   result = minimize(f_runnable, x0, method='Newton-CG', jac=jacobian_f)
   return np.array([*result.x, result.fun, result.nit, result.njev, result.nfev])

# Квази-ньютоновский метод из scipy.optimize с вычислением градиента через разностные схемы
def Quasi_Newton_with_finite_differential_scheme_from_scipy_optimize(f_runnable, x0, eps):
    result = minimize(f_runnable, x0, method='BFGS')
    return np.array([*result.x, result.fun, result.nit, result.njev, result.nfev])

def Quasi_Newton_with_analitical_grad_from_scipy_optimize(f_runnable, f_grad, x0, eps):
    result = minimize(f_runnable, x0, method='BFGS', jac=f_grad)
    return np.array([*result.x, result.fun, result.nit, result.njev, result.nfev])

# Метод Ньютона с одномерным поиском (правило Вольфе)
def Newton_method_Wolfe_rule(f_runnable, f_sympy, x0, eps):
    return Newton_method(f_runnable, f_sympy, step_Wolfe_rule, x0, eps, 0)


In [84]:
def message(result, len_x):
  print("----- Argument values = [", ", ".join(str(result[i]) for i in range(len_x)), "]")
  print("----- Function result = ", result[len_x])
  print("----- Iteration count = ", result[len_x + 1])
  print("----- Gradient evaluation count = ", result[len_x + 2])
  print("----- Function evaluation count = ", result[len_x + 3])
  print()

def test(start_point, f_runnable, f_sympy, const_step, eps):
  len_x = len(start_point)

  print("Function: " + str(f_sympy))
  print("Starting point: " + str(start_point))
  print("Const step: ", const_step)
  print("Required accuracy : ", eps)
  print()

  constant_step_res = Newton_constant_step_method(f_runnable, f_sympy, start_point, const_step, eps)
  print("Newton method with constant step")
  message(constant_step_res, len_x)

  golden_ratio_res = Newton_golden_ratio_method(f_runnable, f_sympy, start_point, eps)
  print("Newton method with golden ratio")
  message(golden_ratio_res, len_x)

  best_step_res = Newton_best_step_method(f_runnable, f_sympy, start_point, eps)
  print("Newton method with best step function")
  message(best_step_res, len_x)

  scipy_res = Newton__method_from_scipy_optimize(f_runnable, start_point, eps)
  print("Newton method from scipy optimize")
  message(scipy_res, len_x)

def test_main(start_point, f_runnable, f_sympy, const_step, eps, Nelder_Mead=True, gradient=True, Newton=True, Quasi_Newton=True):
  len_x = len(start_point)

  print("Function: " + str(f_sympy))
  print("Starting point: " + str(start_point))
  print("Const step: ", const_step)
  print("Required accuracy : ", eps)
  print()
  if Nelder_Mead:
    print("--Нулевой порядок--")

    nelder_mead_res = Nelder_Mead_from_scipy_optimize(f_runnable, start_point, eps)
    print("Nelder-Mead from scipy optimize")
    message(nelder_mead_res, len_x)

  if gradient:
    print("--Градиент--")

    gradient_golden_ratio_res = gradient_descent_golden_ratio(f_runnable, start_point, eps)
    print("Gradient descent with golden ratio")
    message(gradient_golden_ratio_res, len_x)

  if Newton:
    print("--Реализации метода Ньютона--")

    constant_step_res = Newton_constant_step_method(f_runnable, f_sympy, start_point, const_step, eps)
    print("Newton method with constant step")
    message(constant_step_res, len_x)

    golden_ratio_res = Newton_golden_ratio_method(f_runnable, f_sympy, start_point, eps)
    print("Newton method with golden ratio")
    message(golden_ratio_res, len_x)

    best_step_res = Newton_best_step_method(f_runnable, f_sympy, start_point, eps)
    print("Newton method with best step function")
    message(best_step_res, len_x)

    scipy_res = Newton__method_from_scipy_optimize(f_runnable, start_point, eps)
    print("Newton method from scipy optimize")
    message(scipy_res, len_x)

  if Quasi_Newton:
    print("--Квази-ньютоновские реализации--")

    quasi_newton_scipy_res1 = Quasi_Newton_with_finite_differential_scheme_from_scipy_optimize(f_runnable, start_point, eps)
    print("Quasi-Newton from scipy optimize (finite differential scheme)")
    message(quasi_newton_scipy_res1, len_x)


def test_dop2(start_point, f_runnable, f_sympy, const_step, eps):
  len_x = len(start_point)
  print("Function: " + str(f_sympy))
  print("Starting point: " + str(start_point))
  print("Const step: ", const_step)
  print("Required accuracy : ", eps)
  print()

  Wolfe_rule_res = Newton_method_Wolfe_rule(f_runnable, f_sympy, start_point, eps)
  print("Newton method with Wolfe rule")
  message(Wolfe_rule_res, len_x)

  golden_ratio_res = Newton_golden_ratio_method(f_runnable, f_sympy, start_point, eps)
  print("Newton method with golden ratio")
  message(golden_ratio_res, len_x)

  scipy_res = Newton__method_from_scipy_optimize(f_runnable, start_point, eps)
  print("Newton method from scipy optimize")
  message(scipy_res, len_x)


In [85]:
func1_s = x ** 4 + y ** 2 - x * y

def func1_g(x):
  return [4 * x[0] ** 3 - x[1], -x[0] + 2 * x[1]]

def func1_r(x):
  return x[0] ** 4 + x[1] ** 2 - x[0] * x[1]

test_main([-10, 10], func1_r, func1_s, 0.3, 1e-6)

Function: x**4 - x*y + y**2
Starting point: [-10, 10]
Const step:  0.3
Required accuracy :  1e-06

--Нулевой порядок--
Nelder-Mead from scipy optimize
----- Argument values = [ -0.35355178412397703, -0.17675569473594704 ]
----- Function result =  -0.015624999590777656
----- Iteration count =  58.0
----- Gradient evaluation count =  0.0
----- Function evaluation count =  110.0

--Градиент--
Gradient descent with golden ratio
----- Argument values = [ 0.3541794983286121, 0.17771121806651025 ]
----- Function result =  -0.015624417423695733
----- Iteration count =  27.0
----- Gradient evaluation count =  27.0
----- Function evaluation count =  891.0

--Реализации метода Ньютона--
Newton method with constant step
----- Argument values = [ -0.355235853687952, -0.177616821739200 ]
----- Function result =  -0.0156235779145158
----- Iteration count =  91
----- Gradient evaluation count =  91.0
----- Function evaluation count =  182.0

Newton method with golden ratio
----- Argument values = [ -0

Скорость примерно совпадает, как и точность

In [86]:
# Функция Розенброка
func2_s = (1 - x) ** 2 + 100 * (y - x ** 2) ** 2

def func2_r(x):
  return (1 - x[0]) ** 2 + 100 * (x[1] - x[0] ** 2) ** 2

test_main([-100, 10], func2_r, func2_s, 0.3, 1e-6, gradient=False)

Function: (1 - x)**2 + 100*(-x**2 + y)**2
Starting point: [-100, 10]
Const step:  0.3
Required accuracy :  1e-06

--Нулевой порядок--
Nelder-Mead from scipy optimize
----- Argument values = [ 0.9999869896858198, 0.9999708843923565 ]
----- Function result =  1.1272627305581398e-09
----- Iteration count =  155.0
----- Gradient evaluation count =  0.0
----- Function evaluation count =  289.0

--Реализации метода Ньютона--
Newton method with constant step
----- Argument values = [ 0.998643051507238, 0.997251492692670 ]
----- Function result =  1.97418135240291e-6
----- Iteration count =  951
----- Gradient evaluation count =  951.0
----- Function evaluation count =  1902.0

Newton method with golden ratio
----- Argument values = [ 0.999388375216632, 0.998768662577219 ]
----- Function result =  3.81245320043937e-7
----- Iteration count =  431
----- Gradient evaluation count =  431.0
----- Function evaluation count =  14223.0

Newton method with best step function
----- Argument values = [ 0

Первые две реализации метода Ньютона гораздо медленнее третьей, а библиотечная вообще не попадает. Но если поменять стартовую точку на другую относительно банана, то получится

In [87]:
# Функция Розенброка
func2_s = (1 - x) ** 2 + 100 * (y - x ** 2) ** 2

def func2_r(x):
  return (1 - x[0]) ** 2 + 100 * (x[1] - x[0] ** 2) ** 2

test_main([10, -10], func2_r, func2_s, 0.3, 1e-6, gradient=False)

Function: (1 - x)**2 + 100*(-x**2 + y)**2
Starting point: [10, -10]
Const step:  0.3
Required accuracy :  1e-06

--Нулевой порядок--
Nelder-Mead from scipy optimize
----- Argument values = [ 1.0000236025024942, 1.000045308074558 ]
----- Function result =  9.171240085468826e-10
----- Iteration count =  86.0
----- Gradient evaluation count =  0.0
----- Function evaluation count =  162.0

--Реализации метода Ньютона--
Newton method with constant step
----- Argument values = [ 1.00110718085400, 1.00216724475841 ]
----- Function result =  1.45955206532435e-6
----- Iteration count =  243
----- Gradient evaluation count =  243.0
----- Function evaluation count =  486.0

Newton method with golden ratio
----- Argument values = [ 0.999980277504155, 0.999951591864825 ]
----- Function result =  8.42346826275461e-9
----- Iteration count =  93
----- Gradient evaluation count =  93.0
----- Function evaluation count =  3069.0

Newton method with best step function
----- Argument values = [ 0.999700064

Все попали, но третья реализация была быстрее всех. Точность +- одинаковая

In [88]:
func3_s = -((1/(x ** 2 + 0.3)) ** 2 + (1/(y ** 2 + 0.3)) ** 2)

def func3_r(x):
  return -((1/(x[0] ** 2 + 0.3)) ** 2 + (1/(x[1] ** 2 + 0.3)) ** 2)

test_main([-10, 10], func3_r, func3_s, 0.3, 1e-6)

Function: -1/(y**2 + 0.3)**2 - 1/(x**2 + 0.3)**2
Starting point: [-10, 10]
Const step:  0.3
Required accuracy :  1e-06

--Нулевой порядок--
Nelder-Mead from scipy optimize
----- Argument values = [ -5.473005825176055e-06, -0.00022611300968056224 ]
----- Function result =  -22.222218432812305
----- Iteration count =  78.0
----- Gradient evaluation count =  0.0
----- Function evaluation count =  144.0

--Градиент--
Gradient descent with golden ratio
----- Argument values = [ -10.0, 10.0 ]
----- Function result =  -0.00019880537848070942
----- Iteration count =  1.0
----- Gradient evaluation count =  1.0
----- Function evaluation count =  33.0

--Реализации метода Ньютона--
Newton method with constant step
----- Argument values = [ -25.4432727951513, 25.4432650993168 ]
----- Function result =  -4.76799654679420e-6
----- Iteration count =  33
----- Gradient evaluation count =  33.0
----- Function evaluation count =  66.0

Newton method with golden ratio
----- Argument values = [ -10.0, 10.

Тут наоборот самописные методы промахиваются, а библиотечный хорошо себя чувствует

# Выводы

1. В среднем метод Ньютона с шагом
\begin{align}
\alpha = - \frac {grad(f_{(k)})^{Τ} \cdot p_{(k)}} { p_{(k)}^{T} \cdot hessian(f_{(k)}) \cdot p_{(k)}}
\end{align}
работает лучше других наших реализаций: количество итераций и вычислений функции, производных меньше, в некоторых случаях даже на порядок.
2. Существуют функции, для некоторых точек которых, не справляется либо наша реализация, либо библиотечная. Мы не смогли объяснить природу этих результатов, но пришли к выводу, что при необходимости найти минимум стоит использовать несколько реализаций.


# Допольнительное задание 1

Задача: Реализовать одномерный поиск по правилу Вольфе (Wolfe) и метод Ньютона на его
основе и исследовать на эффективность (сравнить с реализованным методами Ньютона и методом Newton-CG).

In [89]:
# Дополнительное задание

func1_s = (1 - x) ** 2 + 100 * (y - x ** 2) ** 2
def func1_r(x):
  return (1 - x[0]) ** 2 + 100 * (x[1] - x[0] ** 2) ** 2

test_dop2([10, -10], func1_r, func1_s, 0.5, 1e-9)

Function: (1 - x)**2 + 100*(-x**2 + y)**2
Starting point: [10, -10]
Const step:  0.5
Required accuracy :  1e-09

Newton method with Wolfe rule
----- Argument values = [ 0.999699647202244, 0.999398884082153 ]
----- Function result =  9.02368565620529e-8
----- Iteration count =  105
----- Gradient evaluation count =  270.0
----- Function evaluation count =  480.0

Newton method with golden ratio
----- Argument values = [ 0.999980300056983, 0.999951658974757 ]
----- Function result =  8.38317879384241e-9
----- Iteration count =  93
----- Gradient evaluation count =  93.0
----- Function evaluation count =  4464.0

Newton method from scipy optimize
----- Argument values = [ 0.9999913347150167, 0.9999826349753403 ]
----- Function result =  7.520639441408052e-11
----- Iteration count =  41.0
----- Gradient evaluation count =  139.0
----- Function evaluation count =  50.0



In [90]:
func5_s = (x - 1) * (x - 3) * (x - 5) * (x - 8) / 100 + y ** 2
def func5_r(X):
    return (X[0] - 1) * (X[0] - 3) * (X[0] - 5) * (X[0] - 8) / 100 + X[1] * X[1]

test_dop2([3, -10], func5_r, func5_s, 0.01, 1e-9)

Function: y**2 + (x - 8)*(x - 5)*(x - 3)*(x - 1)/100
Starting point: [3, -10]
Const step:  0.01
Required accuracy :  1e-09

Newton method with Wolfe rule
----- Argument values = [ 4.03830052664243, -2.60999966054070e-5 ]
----- Function result =  0.120191764680093
----- Iteration count =  17
----- Gradient evaluation count =  50.0
----- Function evaluation count =  84.0

Newton method with golden ratio
----- Argument values = [ 1.77633647374038, -5.00350038857234e-7 ]
----- Function result =  -0.190593381958226
----- Iteration count =  5
----- Gradient evaluation count =  5.0
----- Function evaluation count =  240.0

Newton method from scipy optimize
----- Argument values = [ 1.7763364813673486, -9.219441067073002e-13 ]
----- Function result =  -0.1905933819584762
----- Iteration count =  6.0
----- Gradient evaluation count =  18.0
----- Function evaluation count =  8.0



# Вывод
Метод Ньютона с правилом Вольфе работает примерно аналогично методу Ньютона с золотым сечением: они оба ищут оптимальную длину шага, с которым двигаться по направлению убывания. Поэтому у них аналогичные траектории и примерно одинаковое количество итераций. Однако, во-первых, количество вычислений функции при методе золотого сечения на порядок больше, чем при правиле Вольфе; но количество вычислений градиента при методе золотого сечения в среднем в три раза меньше, чем чем при правиле Вольфе. Во-вторых, обе реализации проигрывают по всем параметрам библиотечной реализации. Итого, стоит понимать, какое из вычисление более трудозатратное – вычисление самой функции или вичисление ее производных, и выбирать соответствующий метод поиска.

# Дополнительное задание 2
В реализацию метода Ньютона была добавлена проверка на нулевой определитель гессиана. В этом случае выводится сообщение об ошибке и вычисление продолжается с помощью градиентного спуска.

In [91]:
# Функция для дополнительного задания 2, часть 1
func_dop2_1_s = (x + y) ** 2
def func_dop2_1_r(x):
  return (x[0] + x[1]) ** 2

test([10, 10], func_dop2_1_r, func_dop2_1_s, 0.5, 1e-6)

Function: (x + y)**2
Starting point: [10, 10]
Const step:  0.5
Required accuracy :  1e-06

Erorr: Matrix det == 0. Current function:  (x + y)**2 , current arguments:  10 10
Calculations continue using gradient descent.

Newton method with constant step
----- Argument values = [ 1.8424369526925943e-05, 1.8424369526925943e-05 ]
----- Function result =  1.3578295698588691e-09
----- Iteration count =  3.0
----- Gradient evaluation count =  3.0
----- Function evaluation count =  99.0

Erorr: Matrix det == 0. Current function:  (x + y)**2 , current arguments:  10 10
Calculations continue using gradient descent.

Newton method with golden ratio
----- Argument values = [ 1.8424369526925943e-05, 1.8424369526925943e-05 ]
----- Function result =  1.3578295698588691e-09
----- Iteration count =  3.0
----- Gradient evaluation count =  3.0
----- Function evaluation count =  99.0

Erorr: Matrix det == 0. Current function:  (x + y)**2 , current arguments:  10 10
Calculations continue using gradient des